To Do:
- Make work based on jurs list

In [1]:
import pandas as pd
import sys  
sys.path.insert(0, '/lib')
import lib.lit_explorer as lit

In [2]:
jurs = [
        "CA",
        "MI",
        "UT",
        "ME"
        ]

In [3]:
files_df_ut = pd.read_csv("../data/raw/www.utcourts.gov/form_data.csv")
files_df_mi = pd.read_csv("../data/raw/www.courts.michigan.gov/form_data.csv")
files_df_me = pd.read_csv("../data/raw/mjbportal.courts.maine.gov/form_data.csv")
files_df_ca = pd.read_csv("../data/raw/www.courts.ca.gov/form_data.csv")

df = pd.concat([files_df_ut,files_df_mi,files_df_me,files_df_ca],ignore_index=True)
df.head()

,id,jurisdiction,source,title,group,url,filename,downloaded
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11


In [5]:
df["pages"] = ""
df["fields"] = ""
df["fields_conf"] = ""
df["fields_old"] = ""
df["f_per_p"] = ""
df["reading"] = ""
df["list"] = ""
df["text"] = ""

print(len(df))
i=0
#for index,row in df[:2].iterrows():
for index,row in df.iterrows():
    if (row["pages"] == "") & (row["jurisdiction"] in jurs):
        print("%s*, "%i, end='')
        try:
            stats = lit.parse_form("../data/processed/"+row["source"]+"/forms/"+row["id"]+".pdf",title=row["title"],jur=row["jurisdiction"],cat=row["group"],normalize=1,use_spot=1,rewrite=1)
            df.at[index, 'pages'] = stats["pages"]
            df.at[index, 'fields'] = stats["fields"]
            df.at[index, 'fields_conf'] = stats["fields_conf"]
            df.at[index, 'fields_old'] = stats["fields_old"]
            df.at[index, 'f_per_p'] = stats["avg fields per page"]
            df.at[index, 'reading'] = stats["reading grade level"]
            df.at[index, 'list'] = stats["list"]
            df.at[index, 'text'] = stats["text"]
            #print(index)
        except:
            print("error: "+"../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf")
    else:
        print("%s, "%i, end='')
        
    i+=1

df

3224
0*, 1*, 2*, 3*, 4*, 5*, 6*, 7*, 8*, 9*, 10*, 11*, 12*, 13*, 14*, 15*, 16*, 17*, 18*, 19*, 20*, 21*, 22*, 23*, 24*, 25*, 26*, 27*, 28*, 29*, 30*, 31*, 32*, 33*, 34*, 35*, 36*, 37*, 38*, 39*, 40*, 41*, 42*, 43*, 44*, 45*, 46*, 47*, 48*, 49*, 50*, 51*, 52*, 53*, 54*, 55*, 56*, 57*, 58*, 59*, 60*, 61*, 62*, 63*, 64*, 65*, 66*, 67*, 68*, 69*, 70*, 71*, 72*, 73*, 74*, 75*, 76*, 77*, 78*, 79*, 80*, 81*, 82*, 83*, 84*, 85*, 86*, 87*, 88*, 89*, 90*, 91*, 92*, 93*, 94*, 95*, 96*, 97*, 98*, 99*, 100*, 101*, 102*, 103*, 104*, 105*, 106*, 107*, 108*, 109*, 110*, 111*, 112*, 113*, 114*, 115*, 116*, 117*, 118*, 119*, 120*, 121*, 122*, 123*, 124*, 125*, 126*, 127*, 128*, 129*, 130*, 131*, 132*, 133*, 134*, 135*, 136*, 137*, 138*, 139*, 140*, 141*, 142*, 143*, 144*, 145*, 146*, 147*, 148*, 149*, 150*, 151*, 152*, 153*, 154*, 155*, 156*, 157*, 158*, 159*, 160*, 161*, 162*, 163*, 164*, 165*, 166*, 167*, 168*, 169*, 170*, 171*, 172*, 173*, 174*, 175*, 176*, 177*, 178*, 179*, 180*, 181*, 182*, 183*, 1

700*, 701*, 702*, 703*, 704*, 705*, 706*, 707*, 708*, 709*, 710*, 711*, 712*, 

713*, 714*, 715*, 

716*, 717*, 718*, 719*, 720*, 721*, 722*, 723*, 724*, 725*, 726*, 727*, 728*, 729*, 730*, 731*, 732*, 733*, 734*, 735*, 736*, 737*, 738*, 739*, 740*, 741*, 742*, 

743*, 744*, 745*, 746*, 747*, 748*, 749*, 750*, 751*, 752*, 753*, 754*, 755*, 756*, 757*, 758*, 759*, 760*, 

...

1666*, 1667*, 1668*, 1669*, 1670*, 1671*, 1672*, 1673*, 1674*, 1675*, 1676*, 1677*, 1678*, 1679*, 1680*, 1681*, 1682*, 1683*, 1684*, 1685*, 1686*, 1687*, 1688*, 1689*, 1690*, 1691*, 1692*, 1693*, 1694*, 1695*, 1696*, 1697*, 1698*, 1699*, 1700*, 1701*, 1702*, 1703*, 1704*, 1705*, 1706*, 1707*, 1708*, 1709*, 1710*, 1711*, 

1712*, 1713*, 1714*, 1715*, 1716*, 1717*, 1718*, 1719*, 1720*, 1721*, 1722*, 1723*, 1724*, 1725*, 1726*, 1727*, 1728*, 1729*, 1730*, 1731*, 1732*, 

1733*, 1734*, 1735*, 1736*, 1737*, 1738*, 1739*, 1740*, 1741*, 1742*, 1743*, 1744*, 1745*, 1746*, 1747*, 1748*, 1749*, 1750*, 1751*, 1752*, 1753*, Timed out!
1754*, 1755*, 1756*, 1757*, 1758*, 1759*, 1760*, 1761*, 1762*, 1763*, 1764*, 1765*, 1766*, Timed out!
1767*, 1768*, Timed out!
1769*, 1770*, 1771*, 1772*, 1773*, 1774*, error: ../data/raw/www.courts.ca.gov/forms/0779d1041707e4ff0ec111426f11a971.pdf
1775*, 1776*, 1777*, 1778*, 1779*, 1780*, 1781*, 1782*, 1783*, 1784*, 1785*, 1786*, 1787*, 1788*, 1789*, 1790*, 1791*, 1792*, 1793*, 1794*, 1795*, 1796*, 1797*, 1798*, 1799*, 1800*, 1801*, 1802*, 1803*, 1804*, 1805*, 1806*, 1807*, 1808*, 1809*, 1810*, 1811*, 1812*, 1813*, 1814*, 1815*, 1816*, 1817*, 1818*, 1819*, 1820*, 1821*, 1822*, 1823*, Timed out!
1824*, 1825*, Timed out!
1826*, 1827*, 1828*, 1829*, 1830*, 1831*, 1832*, 1833*, 1834*, 1835*, 1836*, 1837*, 1838*, 1839*, 1840*, 1841*, 1842*, 1843*, 1844*, 1845*, 1846*, 1847*, 1848*, 1849*, 1850*, 1851*, 1852*, 1853*, 1854*, 1855*, 1856

2543*, 2544*, 2545*, 2546*, 2547*, 2548*, 2549*, 2550*, 2551*, 2552*, 2553*, 2554*, 2555*, 2556*, 2557*, 2558*, 2559*, 2560*, 2561*, 2562*, 2563*, 2564*, 2565*, 2566*, 2567*, 2568*, 2569*, 2570*, 2571*, 2572*, 2573*, 2574*, 2575*, 2576*, 2577*, 2578*, 2579*, 2580*, 2581*, 2582*, 2583*, 2584*, 2585*, 2586*, 2587*, 2588*, 2589*, 2590*, 2591*, 2592*, 2593*, 2594*, 2595*, 2596*, 2597*, 2598*, 2599*, 2600*, 2601*, 2602*, 2603*, 2604*, 2605*, 2606*, 2607*, 2608*, 2609*, 2610*, 2611*, 2612*, 2613*, 2614*, 2615*, 2616*, 2617*, 2618*, 2619*, 2620*, 2621*, 2622*, 2623*, 2624*, 2625*, 2626*, 2627*, 2628*, 2629*, 2630*, 2631*, 2632*, 2633*, 2634*, 2635*, 2636*, 2637*, 2638*, 2639*, 2640*, 2641*, 2642*, 2643*, 2644*, 2645*, 2646*, 2647*, 2648*, 2649*, 2650*, 2651*, 2652*, 2653*, 2654*, 2655*, 2656*, 2657*, 2658*, 2659*, 2660*, 2661*, 2662*, 2663*, 2664*, 2665*, 2666*, 2667*, 2668*, 2669*, 2670*, 2671*, 2672*, 2673*, 2674*, 2675*, 2676*, 2677*, 2678*, 2679*, 2680*, 2681*, 2682*, 2683*, 2684*, 2685*,

,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,fields_conf,fields_old,f_per_p,reading,list,text
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11,2,"[name__1, name__2, *docket_number, *users1_add...","[0.63, 0.61, 1, 1, 1, 0.61, 0.52, 0.53, 0.6, 0...","[name__1, name__2, docket_number, users1_addre...",7.5,14.5,[],COMMUNITY SERVICE WORKSHEET\n THIRD DISTRICT J...
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11,2,"[*docket_number, *users1_birthdate, male, juve...","[1, 1, 0.69, 0.61, 1, 1, 1, 0.56, 0.68, 0.67, ...","[docket_number, users1_birthdate, male, juveni...",19.0,12.5,[GO-00-00-00-00],THIRD DISTRICT JUVENILE COURT WORK PROGRAM REF...
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11,5,"[probation_department_order_court, special_con...","[0.73, 0.65, 0.7, 0.54, 0.68, 0.67, 0.62, 0.59...",[of the probation department or by order of th...,4.8,19.5,[],Page 1 of 4\n UTAH STATE DISTRICT JUVENILE COU...
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11,3,"[*users1_name, *users1_address_line_one, *user...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","[Name, Address, City State Zip, Phone, Email, ...",14.666667,12.5,[],Consent to Adoption and Waiver of \n Rights\n ...
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11,1,"[*users1_name, *users1_address_line_one, *user...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","[Name, Address, City State Zip, Phone, Email, ...",19.0,61.5,[GO-00-00-00-00],Adoption\n \n Agreement\n \n Approved Board of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,6efc9088604befed5fb59fc42aeee0f5,CA,www.courts.ca.gov,Notice of Hearing to Renew Restraining Order,WV,https://www.courts.ca.gov/documents/wv710.pdf,wv710.pdf,2021-11-17,2,"[court_info, case_number, right_caption, full_...","[0.69, 0.56, 0.64, 0.55, 0.55, 0.69, 0.66, 0.6...","[CourtInfo[0], CaseNumber[0], RightCaption[0],...",19.0,12.5,"[CO-07-00-00-00, CO-00-00-00-00]",Notice of Hearing to Renew Restraining Order (...
3220,740afc14f63fc6bf594ee344126ddb54,CA,www.courts.ca.gov,Response to Request to Renew Restraining Order,WV,https://www.courts.ca.gov/documents/wv720.pdf,wv720.pdf,2021-11-17,2,"[court_info, case_number, right_caption, full_...","[0.69, 0.56, 0.64, 0.55, 0.55, 0.69, 0.66, 0.6...","[CourtInfo[0], CaseNumber[0], RightCaption[0],...",22.0,7.5,"[CO-07-00-00-00, CO-00-00-00-00]",Response to Request to Renew Restraining Order...
3221,c3642c835a4c4c0a167fdcc4758b68d9,CA,www.courts.ca.gov,Order Renewing Workplace Violence Restraining ...,WV,https://www.courts.ca.gov/documents/wv730.pdf,wv730.pdf,2021-11-17,1,"[court_info, case_number, right_caption, full_...","[0.69, 0.56, 0.64, 0.55, 0.55, 0.69, 0.66, 0.6...","[CourtInfo[0], CaseNumber[0], RightCaption[0],...",50.0,12.5,[CO-00-00-00-00],Order Renewing Workplace Violence Restraining ...
3222,e2917df3dd9a497daf3a29ca49782672,CA,www.courts.ca.gov,"Proof of Firearms Turned In, Sold, or Stored",WV,https://www.courts.ca.gov/documents/wv800.pdf,wv800.p

In [15]:
#df1 = pd.read_csv("../data/processed/form_data.csv")
#display(df1)
#df2 = df[(df["jurisdiction"]=="CA") & (df["pages"]!="")]
#display(df2)
#dfall = pd.concat([df1,df2],ignore_index=True)
#display(dfall)

In [ ]:
#dfall.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    

In [6]:
df.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    